In [60]:
import csv
import codecs
import collections
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np
from collections import Counter

tokenizer = TweetTokenizer()

In [61]:
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')
data = list(zip(df['text'].tolist(), df['tags'].tolist()))
tags = sorted(set(df['tags'].tolist()))



from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(tags)

LabelEncoder()

In [62]:
tags_cnt = list(Counter([d[1] for d in data]).items())
tags_accepted = [tag[0] for tag in tags_cnt if tag[0] != 'Все' and tag[1] > 100]
print(tags_cnt)
print(tags_accepted)

texts = [d[0] for d in data if d[1] in tags_accepted]
text_tags = le.transform([d[1] for d in data if d[1] in tags_accepted])

[('Катастрофы', 368), ('Бокс и ММА', 1379), ('Искусство', 1495), ('Внешний вид', 211), ('Следствие и суд', 4165), ('Прибалтика', 1043), ('Теннис', 930), ('Архитектура', 232), ('Мемы', 336), ('Футбол', 8678), ('Явления', 1345), ('Экология', 32), ('Наука', 5516), ('Госрегулирование', 41), ('Музыка', 3809), ('Госэкономика', 9190), ('Происшествия', 10858), ('Кино', 6592), ('Криминал', 1497), ('Производители', 36), ('История', 203), ('Мнения', 359), ('Средняя Азия', 936), ('Казахстан', 606), ('Первая мировая', 65), ('Наследие', 14), ('События', 2234), ('Игры', 1954), ('Мир', 1274), ('Москва', 1175), ('Банки', 1876), ('Украина', 13376), ('Книги', 939), ('Общество', 19437), ('Полиция и спецслужбы', 905), ('Авто', 24), ('Автобизнес', 242), ('Культпросвет', 1), ('Финансы компаний', 142), ('Страноведение', 10), ('Технологии', 200), ('Стиль', 1096), ('Туризм', 30), ('Летние виды', 2392), ('Деньги', 987), ('Политика', 22849), ('ТВ и радио', 1951), ('Оружие', 4120), ('Компании', 6626), ('Часы', 355

In [63]:
corpus_size = len(texts)
selections_num = 6      # На сколько частей разбиваем датасет
selection_size = corpus_size // selections_num
print(selection_size)
print(corpus_size - (selections_num - 1) * selection_size)


for i in range(selections_num - 1):
    start = i * selection_size
    end = start + selection_size
    print(str(start) + ' - ' + str(end))

30759
30760
0 - 30759
30759 - 61518
61518 - 92277
92277 - 123036
123036 - 153795


Тренируемся на первых пяти датасетах без лемматизации, проверяемся на шестом

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import pymorphy2 as pm


def check_percentage(i, n):
    r = i / n * 100
    if r > 10 and  r < 10.01:
        print(r)
    elif r > 30 and  r < 30.01:
        print(r)
    elif r > 50 and  r < 50.01:
        print(r)
    elif r > 70 and  r < 70.01:
        print(r)
    elif r > 90 and  r < 90.01:
        print(r)
    

morph = pm.MorphAnalyzer()
stopwords_ru = [w for w in stopwords.words("russian")]

lin_reg_clf = Pipeline([('vect', CountVectorizer(stop_words = stopwords_ru)), 
             ('tfidf', TfidfTransformer()),
             ('clf', LogisticRegression())
               ])
                
lin_reg_models = []

for i in range(selections_num - 1):
    print("Selection #" + str(i))
    start = i * selection_size
    end = start + selection_size
    print("Training logistic regression")
    curr_texts = texts[start:end]
    normalized_texts = []
    
    '''
    n = len(curr_texts)
    for i, text in enumerate(curr_texts):
        check_percentage(i, n)
        tokens = tokenizer.tokenize(text)
        tokens = [morph.parse(t.lower())[0].normal_form for t in tokens if t.isalnum()]
        normalized_texts.append(' '.join(tokens))
        if i % 1000 == 0:
            print("Yay " + str(i))
    print("Done lemmatization")
    '''
    
    lin_reg_models.append(lin_reg_clf.fit(curr_texts, text_tags[start:end]))

Selection #0
Training logistic regression
Selection #1
Training logistic regression
Selection #2
Training logistic regression
Selection #3
Training logistic regression
Selection #4
Training logistic regression


In [65]:
from sklearn.metrics import accuracy_score


start = (selections_num - 1) * selection_size
test_selection_cnt = corpus_size - (selections_num - 1) * selection_size
test_tags = text_tags[start:]

'''
X_test_counts = count_vect.fit(texts[start:])
X_test_tfidf = tfidf_transformer.fit(X_test_counts)
'''

lin_reg_predictions = [model.predict(texts[start:]) for model in lin_reg_models]
prediction = []
#multinom_predictions = [model.predict(X_test_tfidf) for model in multinom_models
for i in range(len(lin_reg_predictions[0])):
    predictions = [p[i] for p in lin_reg_predictions]
    prediction.append(max(set(predictions), key=predictions.count))


prediction = np.array(prediction)
log_reg_accuracy = accuracy_score(test_tags, prediction)
print(log_reg_accuracy)

0.625585175553


# Ниже черновик

In [44]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)

In [7]:
import pandas as pd
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')

In [13]:
df[:3]

,tags,text,title,topic,url
0,Деловой климат,Заместитель председателя правительства Аркадий...,Правительство прокомментировало идею о запрете...,Бизнес,https://lenta.ru/news/2017/04/01/24hours/
1,События,Монреальская конвенция об унификации правил во...,Эксперт отвел год на окончательную ратификацию...,Путешествия,https://lenta.ru/news/2017/03/31/motrealwork/
2,Общество,Сотни байкеров в столице Аргентины Буэнос-Айре...,Аргентинские байкеры устроили акцию протеста п...,Мир,https://lenta.ru/news/2017/03/30/bikers/


In [3]:
# ['tags', 'text', 'title', 'topic', 'url']
tags = [' спецпроект',
 'Coцсети',
 'Авто',
 'Автобизнес',
 'Аналитика рынка',
 'Архитектура',
 'Банки',
 'Баскетбол',
 'Безопасность ',
 'Белоруссия',
 'Бокс и ММА',
 'Вещи',
 'Вирусные ролики',
 'Вкусы',
 'Внешний вид',
 'Вооружение',
 'Вооруженные силы',
 'Все',
 'Выборы',
 'Гаджеты',
 'Госрегулирование',
 'Госэкономика',
 'Движение',
 'Деловой климат',
 'Деньги',
 'Достижения',
 'Еда',
 'Звери',
 'Зимние виды',
 'Игры',
 'Инновации',
 'Инструменты',
 'Интернет',
 'Искусство',
 'История',
 'Кавказ',
 'Казахстан',
 'Катастрофы',
 'Кино',
 'Книги',
 'Компании',
 'Конфликты',
 'Космос',
 'Криминал',
 'Крым',
 'Культпросвет',
 'Легпром',
 'Летние виды',
 'Люди',
 'Мемы',
 'Мир',
 'Мировой бизнес',
 'Мировой опыт',
 'Мнения',
 'Молдавия',
 'Москва',
 'Музыка',
 'Наследие',
 'Наука',
 'Общество',
 'Оружие',
 'Первая мировая',
 'Политика',
 'Полиция и спецслужбы',
 'Пресса',
 'Преступность',
 'Прибалтика',
 'Производители',
 'Происшествия',
 'Регионы',
 'Реклама',
 'Ресурсы',
 'Россия',
 'Рынки',
 'Следствие и суд',
 'События',
 'Средняя Азия',
 'Стиль',
 'Страноведение',
 'ТВ и радио',
 'Театр',
 'Теннис',
 'Техника',
 'Технологии',
 'Туризм',
 'Украина',
 'Финансы компаний',
 'Футбол',
 'Хоккей',
 'Часы',
 'Экология',
 'Явления']

num_tags = [{'tag': tag, 'id': i} for i, tag in enumerate(tags)]

texts = []
text_tags = []

with codecs.open('D:\Max\docs_cluster\lenta\lenta_data.csv', 'r', 'utf_8_sig') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        texts.append(row[1])
        text_tags.append(next((tag['id'] for tag in num_tags if tag['tag'] == row[0]), None))